In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [4]:
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agentforce 360**: Salesforce's comprehensive platform for building enterprise AI agents, featuring Agentforce Builder for natural language-based agent creation, Agent Script for defining agent behavior, and Agentforce Voice for realistic phone interactions. ([itpro.com](https://www.itpro.com/technology/artificial-intelligence/salesforce-just-launched-a-new-catch-all-platform-to-build-enterprise-ai-agents?utm_source=openai))

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models for any AI agent, supporting integration with existing agents with minimal code modifications. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents, achieving state-of-the-art results among open-source agents. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework that allows users to create and deploy large language model agents through natural language alone, serving as a versatile multi-agent system for general AI assistants. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **Amazon Bedrock AgentCore**: AWS's platform designed to simplify the development and deployment of advanced AI agents, including features like AgentCore Runtime for scalable serverless deployment and AgentCore Memory for context management. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize the integration and sharing of data between AI systems and external tools, adopted by major AI providers including OpenAI and Google DeepMind. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

- **Kruti**: A multilingual AI agent developed by Ola Krutrim, designed to perform real-world tasks for users, such as booking taxis and ordering food, by integrating directly with various online services. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, capable of independent reasoning, dynamic planning, and autonomous decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and platforms for creating intelligent, autonomous systems. 

In [6]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [7]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the latest trends and frameworks for AI agents that are emerging in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To get information on specific tools and technologies that are being used for AI agents this year.', query='top AI agent technologies 2025'), WebSearchItem(reason='To learn about advancements in AI agent frameworks and their applications in various industries in 2025.', query='AI agent frameworks applications 2025')]


In [8]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [9]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [10]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

# async def send_email(report: ReportData):
#     """ Use the email agent to send an email with the report """
#     print("Writing email...")
#     result = await Runner.run(email_agent, report.markdown_report)
#     print("Email sent")
#     return report

In [12]:
query = "Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    print("Report Summary:")
    print(report.short_summary)
    print("\nFull Report (Markdown):")
    print(report.markdown_report)
    print("Hooray!")


Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Report Summary:
In 2025, the landscape of AI agent frameworks has rapidly evolved with significant advancements such as Salesforce's Agentforce 360 for enterprise integration, OpenAI's lightweight Agents SDK for multi-agent workflows, and Google’s modular Agent Development Kit. Key trends include increased autonomy in decision-making, the rise of specialized agents for various industries, and the adoption of standards like the Model Context Protocol to improve interoperability.

Full Report (Markdown):
# Latest AI Agent Frameworks in 2025

## Introduction
The field of Artificial Intelligence (AI) has seen monumental growth and diversification, especially in the realm of AI agents. As we step into 2025, we are witnessing an array of innovative frameworks designed to facilitate the development, deployment, and scaling of AI agents across vario